In [69]:
import os
import pandas as pd
import numpy as np
from scipy.signal import find_peaks, welch  
from scipy.stats import kurtosis, skew


In [70]:

#Helper - ensure time array is numeric (in ms)
def get_numeric_time(time_col):
    # Try converting directly to float (if ms or sec), fallback to datetime
    try:
        # Remove any commas or extra chars, then convert to float
        time_numeric = pd.to_numeric(time_col, errors='raise')
        return time_numeric.values
    except Exception:
        # If conversion fails, treat as timestamp and convert to ms elapsed
        time_dt = pd.to_datetime(time_col)
        time_numeric = (time_dt - time_dt.iloc[0]).dt.total_seconds() * 1000
        return time_numeric.values

In [71]:
# Time-domain feature extraction
def extract_time_domain_features(signal, time):
    features = {}
    time = np.array(time, dtype=float)

    peaks, _ = find_peaks(signal)
    troughs, _ = find_peaks(-signal)

    def time_to_index(target_ms):
        return np.argmin(np.abs(time - target_ms))

    # N35: Min in 25–45 ms
    n35_window = (time_to_index(25), time_to_index(45))
    n35_idx = troughs[(troughs >= n35_window[0]) & (troughs <= n35_window[1])]
    n35_amp = signal[n35_idx].min() if len(n35_idx) > 0 else np.nan
    features['N35_amp'] = n35_amp
    features['N35_ms'] = time[n35_idx].min() if len(n35_idx) > 0 else np.nan

    # P50: Max in 40–60 ms
    p50_window = (time_to_index(40), time_to_index(60))
    p50_idx = peaks[(peaks >= p50_window[0]) & (peaks <= p50_window[1])]
    p50_amp = signal[p50_idx].max() if len(p50_idx) > 0 else np.nan
    features['P50_amp'] = p50_amp
    features['P50_ms'] = time[p50_idx].min() if len(p50_idx) > 0 else np.nan

    # N95: Min in 85–115 ms
    n95_window = (time_to_index(85), time_to_index(115))
    n95_idx = troughs[(troughs >= n95_window[0]) & (troughs <= n95_window[1])]
    n95_amp = signal[n95_idx].min() if len(n95_idx) > 0 else np.nan
    features['N95_amp'] = n95_amp
    features['N95_ms'] = time[n95_idx].min() if len(n95_idx) > 0 else np.nan

    # Ratios
    if not np.isnan(features['N95_amp']) and not np.isnan(features['P50_amp']):
        features['N95P50_ratio'] = abs(features['N95_amp']) / abs(features['P50_amp'])
    else:
        features['N95P50_ratio'] = np.nan

    return features


In [72]:
def extract_statistical_features(signal):
    features = {}
    features['mean'] = np.mean(signal)
    features['std'] = np.std(signal)
    features['min'] = np.min(signal)
    features['max'] = np.max(signal)
    features['median'] = np.median(signal)
    features['ptp'] = np.ptp(signal)  # Peak-to-peak amplitude
    features['rms'] = np.sqrt(np.mean(signal**2))  # Root mean square
    features['kurt'] = kurtosis(signal)  # Kurtosis
    features['skew'] = skew(signal)  # Skewness
    features['zcr'] = ((signal[:-1] * signal[1:]) < 0).sum() / len(signal)  # Zero-crossing rate
    return features

In [73]:
def extract_frequency_features(signal, fs):
    features = {}
    f, Pxx = welch(signal, fs=fs, nperseg=256)
    peak_freq = f[np.argmax(Pxx)]
    total_power = np.sum(Pxx)

    # Band powers (e.g. bp_1_30, bp_8_16)
    bp_1_30 = np.trapz(Pxx[(f >= 1) & (f <= 30)], f[(f >= 1) & (f <= 30)])
    bp_8_16 = np.trapz(Pxx[(f >= 8) & (f <= 16)], f[(f >= 8) & (f <= 16)])

    features['peak_freq'] = peak_freq
    features['total_power'] = total_power
    features['bp_1_30'] = bp_1_30
    features['bp_8_16'] = bp_8_16
    return features

In [74]:
#Paths and parameters
processed_data_path = 'D:/perg/data/processed/all_processed'
fs = 1700  # Hz

In [75]:
features_list = []
for file in os.listdir(processed_data_path):
    if file.endswith('.csv'):
        record_id = file.replace('.csv', '')
        df = pd.read_csv(os.path.join(processed_data_path, file))

        # Try to detect the time column dynamically
        time_col = None
        for col in df.columns:
            if col.startswith("TIME_"):   # TIME_1, TIME_2, etc.
                time_col = col
                break

        if time_col is not None:
            time = get_numeric_time(df[time_col])
        else:
            # fallback: make a synthetic time vector if no TIME_* column
            time = np.arange(len(df))

        # Loop through all eyes dynamically
        eye_cols = [c for c in df.columns if c.endswith('_processed')]
        for eye in eye_cols:
            signal = df[eye].values
            time_features = extract_time_domain_features(signal, time)
            stat_features = extract_statistical_features(signal)
            freq_features = extract_frequency_features(signal, fs)
            combined = {
                'record_id': record_id,
                'eye': eye[:2]  # 'RE' or 'LE'
            }
            combined.update(time_features)
            combined.update(stat_features)
            combined.update(freq_features)
            features_list.append(combined)


C:\Users\VICTUS\AppData\Local\Temp\ipykernel_22056\700452499.py:3: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 255, using nperseg = 255
  f, Pxx = welch(signal, fs=fs, nperseg=256)
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_22056\700452499.py:3: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 255, using nperseg = 255
  f, Pxx = welch(signal, fs=fs, nperseg=256)
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_22056\700452499.py:3: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 255, using nperseg = 255
  f, Pxx = welch(signal, fs=fs, nperseg=256)
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_22056\700452499.py:3: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 255, using nperseg = 255
  f, Pxx = welch(signal, fs=fs, nperseg=256)
C:\Users\VICTUS\AppData\Local\Temp\ipykernel_22056\700452499.py:3: UserWarning: nperseg=256 is greater than signal length max(len(x), len(y)) = 

In [76]:
#  Save features
features_df = pd.DataFrame(features_list)
os.makedirs('../outputs/features', exist_ok=True)
features_df.to_csv('../outputs/features/perg_features.csv', index=False)
print("Feature extraction complete. Saved to ../outputs/features/perg_features.csv")
features_df.head()

Feature extraction complete. Saved to ../outputs/features/perg_features.csv


,record_id,eye,N35_amp,N35_ms,P50_amp,P50_ms,N95_amp,N95_ms,N95P50_ratio,mean,...,median,ptp,rms,kurt,skew,zcr,peak_freq,total_power,bp_1_30,bp_8_16
0,0001,RE,-0.408717,28.9,2.199887,52.5,-1.716555,91.5,0.780293,5.572884e-17,...,-0.032117,3.916442,1.0,-0.217159,0.453907,0.015686,13.333333,0.241531,1.207930,0.0
1,0001,LE,-0.712719,29.5,2.394248,53.7,-1.588476,92.1,0.663455,-1.114577e-16,...,-0.236019,3.982724,1.0,0.084987,0.844791,0.011765,13.333333,0.237099,1.250359,0.0
2,0002,RE,-0.045393,31.9,1.063206,46.6,-1.188255,111.6,1.117615,-5.572884e-17,...,0.088699,3.294507,1.0,-1.371636,-0.063129,0.011765,6.666667,0.083373,0.360900,0.0
3,0002,LE,NaN,NaN,NaN,NaN,-1.327168,92.1,NaN,2.716781e-16,...,-0.414977,3.674116,1.0,-0.869275,0.661798,0.007843,6.666667,0.105886,0.245580,0.0
4,0002,RE,-1.622818,42.5,NaN,NaN,-1.363470,102.1,NaN,2.786442e-16,...,-0.168620,4.387108,1.0,0.445414,0.855903,0.031373,13.333333,0.086280,0.320347,0.0
